In [1]:
from astropy.io import fits
from astropy.visualization.lupton_rgb import make_lupton_rgb
import numpy as np
from glob import glob
import os.path

In [2]:
def crops(fnames, xrange, yrange):
    return [fits.open(f)[0].data[xrange[0]:xrange[1], yrange[0]:yrange[1]] for f in fnames]

In [3]:
def multicrops(field, out='crops', cropsize=256,
               scales=(1, 1.3, 7.0),
               minimum=-0.01, stretch=0.05, Q=8,
               datapath='/gpfs01/home/ppzsb1/astobjdet/images',
               bands=['814', '606', '435'],
               maskband=['814']):
    fr, fg, fb = [glob(os.path.join(datapath, field, 'hlsp*{}*drz.fits'.format(b)))[0] for b in bands]
    fm = glob(os.path.join(datapath, field, '*{}*seg.fits'.format(maskband)))[0]
    out = os.path.join(datapath, field, out)
    os.makedirs(out, exist_ok=True)
    with fits.open(fm) as mask:
        shape = mask[0].shape
    # figure out how many crops fit into image
    # then choose limits to give even border
    # not going to bother with overlaps now we have more images
    nx = int(shape[0] / cropsize)
    ny = int(shape[1] / cropsize)
    px = nx * cropsize
    py = ny * cropsize
    bx = int((shape[0] - px) / 2)
    by = int((shape[1] - py) / 2)
    xl, xh = bx, bx + px
    yl, yh = by, by + py
    xranges = list(zip(np.arange(xl, xh+1, cropsize)[:-1],
                       np.arange(xl, xh+1, cropsize)[1:]))
    yranges = list(zip(np.arange(yl, yh+1, cropsize)[:-1],
                       np.arange(yl, yh+1, cropsize)[1:]))
    for i, xr in enumerate(xranges):
        for j, yr in enumerate(yranges):
            r, g, b, m = crops([fr, fg, fb, fm], xr, yr)
            if m.sum() > 0:
                # if there are some objects in the mask
                img = make_lupton_rgb(r * scales[0],
                                      g * scales[1],
                                      b * scales[2],
                                      minimum=minimum, stretch=stretch, Q=Q)
                np.save(os.path.join(out, 'crop_{}_{}.npy'.format(i, j)), img)
                np.save(os.path.join(out, 'mask_{}_{}.npy'.format(i, j)), m)

In [ ]:
fields = ['abell2744', 'abell2744par',
          'abell370', 'abell370par',
          'abells1063', 'abells1063par',
          'macs0416', 'macs0416par',
          'macs0717', 'macs0717par',
          'macs1149', 'macs1149par']

for f in fields:
    multicrops(f, out='crops', minimum=-0.01, stretch=0.1, Q=4)